In [1]:
import glob
import pandas as pd
import numpy as np

from deepdiff import DeepDiff

In [2]:
def to_config(line,access):
    return eval(line.replace(" ".join(line.split(' ')[:4]),'').replace(f'%s Mismatches'%access,"{'%s'"%access)+"}")

def get_registers(idict):
    parameters = []
    for access,accessDict in idict.items():
        if 'ASIC' in accessDict.keys():
            accessDict = accessDict['ASIC']
        for block,blockDict in accessDict.items():
            for param, paramDict in blockDict.items():
                parameters.append(f'{access}_{block}_{param}')
    return parameters

            
RW_hardreset = {}
RO_hardreset = {}
with open('logs/Monitoring_logs/log_March_12_InBeam_ForcedReset.txt', 'r') as f:
    for line in f:
        if 'RW Mismatches' in line and not RW_hardreset:
            RW_hardreset = to_config(line,'RW')
        if 'RO Mismatches' in line and not RW_hardreset:
            RO_hardreset = to_config(line,'RO')

logList=glob.glob('logs/Monitoring_logs/log_March_*.txt')


In [3]:
# ##
# ## Original setup from Cristina (small bug when "Resync Time" falls at certain spots, where you are looking for a message in the next line as well)
# to_read = [
#     'n_OOT_errors',
#     'n_IT_errors',
#     'n_RW_mismatches',
#     'n_RO_mismatches',
#     'hardresets',
#     'RW_mismatches',
#     'RO_mismatches',
#     'RO_aligner_mismatches'
#     'start_time',
#     'stop_time',
#     'start_time_OOT',
#     'stop_time_OOT',
#     'log_name',
#     'netx',
#     'algorithm',
# ]

# monitoring_logs = {}

# for log in logList:
#     file1 = open(log, 'r')
#     lines = file1.readlines()
#     key = ""
#     dataset = ""
#     netx = 13
#     algo = 0
#     for i,line in enumerate(lines):
#         try:
#             if 'csv file' in line:
#                 dataset = line.split(' ')[-1]
#             elif 'Initial RW Status' in line:
#                 config = eval(lines[i+1].replace(" ".join(lines[i+1].split(' ')[:4]),''))
#                 netx = int(config['FMTBUF_ALL']['config_eporttx_numen'])
#                 algo = int(config['MFC_ALGORITHM_SEL_DENSITY']['algo_select'])
#             elif 'Starting data comparisons' in line and 'PULSE IS COMING' in lines[i+1]:
#                 key = ' '.join(lines[i+1].split(' ')[:2]) # take timestamp from pulse
#                 monitoring_logs[key] = dict.fromkeys(to_read)
#                 monitoring_logs[key]['start_time'] = ' '.join(line.split(' ')[:2])
#                 monitoring_logs[key]['log_name'] = log
#                 monitoring_logs[key]['dataset'] = dataset.split('/')[-2]
#                 monitoring_logs[key]['netx'] = netx
#                 monitoring_logs[key]['algorithm'] = algo
#             elif 'Stopping comparisons' in line and 'Error counter' in lines[i+1]:
#                 monitoring_logs[key]['stop_time'] = ' '.join(line.split(' ')[:2])
#                 monitoring_logs[key]['n_IT_errors'] = int(lines[i+1].split('\n')[0].split(' ')[-1])
#             elif 'Stopping OOT' in line and 'Error counter' in lines[i+1]:
#                 monitoring_logs[key]['stop_time_OOT'] = ' '.join(line.split(' ')[:2])
#                 monitoring_logs[key]['n_OOT_errors'] = int(lines[i+1].split('\n')[0].split(' ')[-1])
#             elif 'Starting OOT' in line:
#                 monitoring_logs[key]['start_time_OOT'] = ' '.join(line.split(' ')[:2])
#             elif 'RW Mismatches' in line:
#                 config = to_config(line,'RW')
#                 reg = get_registers(config)
#                 monitoring_logs[key]['n_RW_mismatches'] = len(reg)
#                 if config==RW_hardreset:
#                     monitoring_logs[key]['hardresets'] = 1
#                     monitoring_logs[key]['RW_mismatches'] = []    
#                 else:
#                     ddiff = DeepDiff(config, RW_hardreset, ignore_order=True)
#                     if 'iterable_item_removed' not in ddiff.keys():
#                         monitoring_logs[key]['hardresets'] = 1
#                         monitoring_logs[key]['RW_mismatches'] = [] 
#                     else:
#                         monitoring_logs[key]['hardresets'] = None
#                         monitoring_logs[key]['RW_mismatches'] = reg
#             elif 'ERROR  RO Mismatches' in line:
#                 config = get_registers(to_config(line,'RO'))
#                 monitoring_logs[key]['n_RO_mismatches'] = len(config)
#                 if monitoring_logs[key]['hardresets'] == 1:
#                     monitoring_logs[key]['RO_mismatches'] = []
#                 else:
#                     monitoring_logs[key]['RO_mismatches'] = config
#                 #print(key,monitoring_logs[key]['RO_mismatches'])
#             elif 'DEBUG  RO Mismatches' in line:
#                 config = get_registers(to_config(line,'RO'))
#                 if monitoring_logs[key]['hardresets'] == 1:
#                     monitoring_logs[key]['RO_aligner_mismatches'] = []
#                 else:
#                     monitoring_logs[key]['RO_aligner_mismatches'] = config
#         except:
#             print('Could not parse ',log)
#             print(i, line)
# #             break
            
# config = {
#     'PulseTime': [],
#     'StartTime': [],
#     'StopTime': [],
#     'StartTimeOOT': [],
#     'StopTimeOOT': [], 
#     'nOOT': [],
#     'nIT': [],
#     'HardReset': [],
#     'nRW': [],
#     'nRO': [],
#     'RW': [],
#     'RO': [],
#     'ROaligner': [],
#     'Log': [],
#     'Dataset': [],
#     'neTx': [],
#     'Algo': [],
# }
# for key,item in monitoring_logs.items():
#     config['PulseTime'].append(key)
#     config['StartTime'].append(item['start_time'])
#     config['StopTime'].append(item['stop_time'])
#     config['StartTimeOOT'].append(item['start_time_OOT'])
#     config['StopTimeOOT'].append(item['stop_time_OOT'])
#     config['HardReset'].append(item['hardresets'] if item['hardresets'] else 0)
#     config['nRW'].append(item['n_RW_mismatches'] if item['n_RW_mismatches'] else 0)
#     config['nRO'].append(item['n_RO_mismatches'] if item['n_RO_mismatches'] else 0)
#     config['nOOT'].append(item['n_OOT_errors'] if item['n_OOT_errors'] else 0)
#     config['nIT'].append(item['n_IT_errors'] if item['n_IT_errors'] else 0)
#     config['RW'].append(item['RW_mismatches'] if item['RW_mismatches'] else [])
#     config['RO'].append(item['RO_mismatches'] if item['RO_mismatches'] else [])
#     config['ROaligner'].append(item['RO_aligner_mismatches'] if 'RO_aligner_mismatches' in item else [])
#     config['Log'].append(item['log_name'])
#     config['Dataset'].append(item['dataset'])
#     config['neTx'].append(item['netx'])
#     config['Algo'].append(item['algorithm'])

# df=pd.DataFrame.from_dict(config)
# df.set_index('PulseTime',inplace=True)
# df.sort_index(inplace=True)


In [4]:
###
### Different implementation by Danny, based on Cristina's setup

outputs=[]
for log in logList[:]:
    file1 = open(log, 'r')
    lines = file1.readlines()
    key = ""
    netx = 13
    algo = 0
    
    dataset = ""
    netx=13
    algo=0
    log_name=log

    pulseTime=None
    compareStopTime=None
    ootCompareStopTime=None
    nErr=None
    nErrOOT=None
    hardreset=None
    nRWmismatches=None
    RWmismatches=None
    nROmismatches=None
    ROmismatches=None
    ROalignermismatches=None    
    
    for i,line in enumerate(lines):
        try:
            if line.startswith('#'): #add in a comment character option, to allow commenting out lines to edit logs
                continue
            if 'csv file' in line:
                dataset = line.split(' ')[-1]
            elif 'Initial RW Status' in line:
                #if a new run started, reset values
                pulseTime=None
                compareStopTime=None
                ootCompareStopTime=None
                nErr=None
                nErrOOT=None
                hardreset=None
                nRWmismatches=None
                RWmismatches=None
                nROmismatches=None
                ROmismatches=None
                ROalignermismatches=None    
                
                config = eval(lines[i+1].replace(" ".join(lines[i+1].split(' ')[:4]),''))
                netx = int(config['FMTBUF_ALL']['config_eporttx_numen'])
                algo = int(config['MFC_ALGORITHM_SEL_DENSITY']['algo_select'])
            elif 'Starting data comparisons' in line or 'INFO   Closing' in line: 
                #add to output at beginning of next spill period 
                #also add whenever exiting (to capture incomplete spill information)
                if (not pulseTime is None) and (not compareStopTime is None):
                    #(using pulseTime=None to indicate you haven't gone through one yet)
                    outputs.append([pulseTime,
                                    compareStopTime,
                                    ootCompareStopTime,
                                    nErr,
                                    nErrOOT,
                                    hardreset,
                                    nRWmismatches,
                                    nROmismatches,
                                    RWmismatches,
                                    ROmismatches,
                                    ROalignermismatches,
                                    log_name,
                                    dataset,
                                    netx,
                                    algo
                                   ])
                pulseTime=None
                compareStopTime=None
                ootCompareStopTime=None
                nErr=None
                nErrOOT=None
                hardreset=None
                nRWmismatches=None
                RWmismatches=None
                nROmismatches=None
                ROmismatches=None
                ROalignermismatches=None    

            elif 'PULSE IS COMING' in line:
                pulseTime = ' '.join(line.split(' ')[:2]) # take timestamp from pulse
            elif 'Stopping comparisons' in line and 'Error counter' in lines[i+1]:
                compareStopTime = ' '.join(line.split(' ')[:2])
                nErr = int(lines[i+1].split('\n')[0].split(' ')[-1])
            elif 'Stopping OOT' in line and 'Error counter' in lines[i+1]:
                ootCompareStopTime = ' '.join(line.split(' ')[:2])
                nErrOOT = int(lines[i+1].split('\n')[0].split(' ')[-1])
            elif 'RW Matches' in line:
                nRWmismatches=0
                RWmismatches=[]
                hardreset=0
            elif 'RW Mismatches' in line:
                config = to_config(line,'RW')
                reg = get_registers(config)
                nRWmismatches = len(reg)
                if config==RW_hardreset:
                    hardreset = 1
                    RWmismatches = []    
                else:
                    ddiff = DeepDiff(config, RW_hardreset, ignore_order=True)
                    if 'iterable_item_removed' not in ddiff.keys():
                        hardresets = 1
                        RWmismatches = [] 
                    else:
                        hardresets=0
                        RWmismatches = reg
            elif 'RO Matches' in line:
                nROmismatches=0
                ROmismatches=[]
                ROalignermismatches = []
            elif 'ERROR  RO Mismatches' in line:
                config = get_registers(to_config(line,'RO'))
                nROmismatches = len(config)
                if hardreset == 1:
                    ROmismatches = []
                else:
                    ROmismatches = config
            elif 'DEBUG  RO Mismatches' in line:
                config = get_registers(to_config(line,'RO'))
                if hardresets == 1:
                    ROalignermismatches = []
                else:
                    ROalignermismatches = config
        except:
            print('Could not parse ',log)
            print(i, line)
#             break
            

    
    
columns  = ['PulseTime',
            'compareStopTime',
            'ootCompareStopTime',
            'nErr',
            'nErrOOT',
            'HardReset',
            'nRW',
            'nRO',
            'RW',
            'RO',
            'ROaligner',
            'Log',
            'Dataset',
            'neTx',
            'Algo'
           ]

df=pd.DataFrame(outputs,columns=columns)
df.PulseTime=pd.to_datetime(df.PulseTime)
df.compareStopTime=pd.to_datetime(df.compareStopTime)
df.ootCompareStopTime=pd.to_datetime(df.ootCompareStopTime)
df.set_index('PulseTime',inplace=True)
df.sort_index(inplace=True)
df.loc[:,['nErr','nErrOOT','HardReset','nRW','nRO']]=df.loc[:,['nErr','nErrOOT','HardReset','nRW','nRO']].fillna(0).astype(int)

In [5]:
df.to_csv('logs/MonitoringInfo.csv')